In [ ]:
!pip install pycaret

In [ ]:
import pandas as pd
import numpy as np
from category_encoders import *
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
from pycaret.classification import *
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
df_train = pd.read_csv("../input/health-insurance-lead-prediction/train.csv")
df_test = pd.read_csv("../input/health-insurance-lead-prediction/test.csv")

In [ ]:
df_train

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

In [ ]:
for i in df_train.columns:
    print(i,"\n")
    print(df_train[i].value_counts())
    print("Length: ",len(df_train[i].value_counts()))

In [ ]:
df_train.corr()

In [ ]:
def clean_city_code_train(df):
    clean_city_code_train.Mean_encoded_city_code = df.groupby(['City_Code'])['Response'].mean().to_dict() 
    df['City_Code'] = df['City_Code'].map(clean_city_code_train.Mean_encoded_city_code) 
def clean_city_code_test(df):
    for i, j in enumerate(df.City_Code):
        df.City_Code.iloc[i] = clean_city_code_train.Mean_encoded_city_code[j]
    df["City_Code"] = df["City_Code"].astype("float")   

In [ ]:
clean_city_code_train(df_train)
clean_city_code_test(df_test)


In [ ]:
df_train

In [ ]:
df_test

In [ ]:
# def clean_region_code_train(df):
#     clean_region_code_train.Mean_encoded_region_code = df.groupby(['Region_Code'])['Response'].mean().to_dict() 
#     df['Region_Code'] = df['Region_Code'].map(clean_region_code_train.Mean_encoded_region_code) 
# def clean_region_code_test(df):
#     for i, j in enumerate(df.Region_Code):
#         df.Region_Code.iloc[i] = clean_region_code_train.Mean_encoded_region_code[j]

In [ ]:
# clean_region_code_train(df_train)
# clean_region_code_test(df_test)

In [ ]:
# train_reg_code = np.sort(df_train["Region_Code"].unique())
# test_reg_code = np.sort(df_test["Region_Code"].unique())

In [ ]:
# print("Missing values in Test Data from train data:", (set(test_reg_code).difference(train_reg_code))) 

In [ ]:
# plt.xticks(rotation = 90)
# plt.scatter(df_train["City_Code"],df_train["Region_Code"])

In [ ]:
df_train.corr()

In [ ]:
def clean_Accomodation_Type(df):
    df["Accomodation_Type"] = pd.get_dummies(df["Accomodation_Type"],drop_first=True)
def clean_Reco_Insurance_Type(df):
    df["Reco_Insurance_Type"] = pd.get_dummies(df["Reco_Insurance_Type"],drop_first=True)
def clean_Is_Spouse(df):
    df["Is_Spouse"] = pd.get_dummies(df["Is_Spouse"],drop_first=True)

In [ ]:
clean_Accomodation_Type(df_train)
clean_Accomodation_Type(df_test)
clean_Reco_Insurance_Type(df_train)
clean_Reco_Insurance_Type(df_test)
clean_Is_Spouse(df_train)
clean_Is_Spouse(df_test)

In [ ]:
df_train

In [ ]:
def fillna_Health_Indicator(df):
    df.fillna("Not Available",inplace= True)
def fillna_policy(df):
    df.fillna(0,inplace= True)

In [ ]:
fillna_Health_Indicator(df_train["Health Indicator"])
fillna_Health_Indicator(df_test["Health Indicator"])
fillna_policy(df_train["Holding_Policy_Duration"])
fillna_policy(df_test["Holding_Policy_Duration"])
fillna_policy(df_train["Holding_Policy_Type"])
fillna_policy(df_test["Holding_Policy_Type"])

In [ ]:
df_train.isna().sum()

In [ ]:
df_test.isna().sum()

In [ ]:
def one_hot_encoding(df,column):
    encoder=OneHotEncoder(cols=column,return_df=True,use_cat_names=True)
    df_final = encoder.fit_transform(df)
    return df_final

In [ ]:
df_train = one_hot_encoding(df_train,"Health Indicator")
df_train = one_hot_encoding(df_train,"Holding_Policy_Type")
df_test = one_hot_encoding(df_test,"Health Indicator")
df_test = one_hot_encoding(df_test,"Holding_Policy_Type")

In [ ]:
df_train

In [ ]:
df_train.columns

In [ ]:
df_test.columns

In [ ]:
converted_list_1 = []
def clean_Holding_Policy_Duration(df):
    for i in df["Holding_Policy_Duration"]:
        if i == "14+":
            i = 16
            converted_list_1.append(i)
        else:
            pass
            converted_list_1.append(i)
    df["Holding_Policy_Duration"]=pd.Series(converted_list_1)
    df["Holding_Policy_Duration"] = df["Holding_Policy_Duration"].astype("float")

In [ ]:
clean_Holding_Policy_Duration(df_train)
clean_Holding_Policy_Duration(df_test)

In [ ]:
df_train

In [ ]:
df_test

In [ ]:
def clean_Reco_Policy_Cat_train(df):
    clean_Reco_Policy_Cat_train.Mean_encoded_Reco_Policy_Cat = df.groupby(['Reco_Policy_Cat'])['Response'].mean().to_dict() 
    df['Reco_Policy_Cat'] = df['Reco_Policy_Cat'].map(clean_Reco_Policy_Cat_train.Mean_encoded_Reco_Policy_Cat) 
def clean_Reco_Policy_Cat_test(df):
    for i, j in enumerate(df.Reco_Policy_Cat):
        df.Reco_Policy_Cat.iloc[i] = clean_Reco_Policy_Cat_train.Mean_encoded_Reco_Policy_Cat[j]

In [ ]:
clean_Reco_Policy_Cat_train(df_train)
clean_Reco_Policy_Cat_test(df_test)

In [ ]:
def drop_features_train(df):
    df.drop(["ID","Region_Code"],axis = 1,inplace = True)
def drop_features_test(df):
    df.drop(["Region_Code"],axis = 1,inplace = True)

In [ ]:
drop_features_train(df_train)
drop_features_test(df_test)

In [ ]:
X = df_train.drop("Response",axis = 1)
Y = df_train["Response"]

In [ ]:
train,test = train_test_split(df_train, test_size=0.2,random_state= 27)

In [ ]:
experiment = setup(
    data = train,
    target = 'Response',
    normalize = True
)

In [ ]:
compare_models()

In [ ]:
X_train = df_train.drop("Response",axis = 1)
X_test = df_test.drop("ID",axis = 1)
Y_train = df_train["Response"]

In [ ]:
st = StandardScaler()   
X_train= st.fit_transform(X_train)    


In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier= RandomForestClassifier(n_estimators= 10, criterion="entropy")  
classifier.fit(X_train, Y_train)  

In [ ]:
Y_pred= classifier.predict(X_test)  

In [ ]:
submission = pd.DataFrame(df_test['ID'])
submission["Response"] = Y_pred

In [ ]:
submission["Response"].value_counts()